# LiquidityProviders

> It includes three classes of ultra liquidity providers: Central bank, Treasury and Commercial banks

In [ ]:
#| default_exp LiquidityProviders

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
from typing import Dict,List
from pydantic import BaseModel

In [ ]:
#| export

class AccountEntry(BaseModel):
    asset: Dict[str, float]
    liability: Dict[str, float]

class LiquidityProviders():
    def __init__(self,name:str):
        self.name = name   
    def __str__(self):
        return f'{self.name}'
    __repr__=__str__

class BaseLiquidityProvider(LiquidityProviders):
    def __init__(self, name: str, allowed_assets: List[str], allowed_liabilities: List[str], entry: AccountEntry = AccountEntry(asset={}, liability={})):
        super().__init__(name)
        self.asset = {k: 0.0 for k in allowed_assets}
        self.liability = {k: 0.0 for k in allowed_liabilities}

        input_data = {'asset': entry.asset, 'liability': entry.liability}
        validated_data = AccountEntry(**input_data)

        if not set(validated_data.asset.keys()).issubset(set(allowed_assets)):
            raise ValueError(f'Invalid asset item. The valid items are {allowed_assets}.')
        if not set(validated_data.liability.keys()).issubset(set(allowed_liabilities)):
            raise ValueError('Invalid liability item.')
        if sum(validated_data.asset.values()) != sum(validated_data.liability.values()):
            raise ValueError('The sum of asset must be equal to the sum of liability!')

        for k, v in validated_data.asset.items():
            self.asset[k] = v

        for k, v in validated_data.liability.items():
            self.liability[k] = v

    @property
    def total_asset(self):
        return sum(self.asset.values())

    @property
    def total_liability(self):
        return sum(self.liability.values())

    def __str__(self):
        return f'{self.name}: total asset/liability is {self.total_asset}'

    __repr__ = __str__


class CentralBank(BaseLiquidityProvider):
    allowed_assets = ['Treasury', 'CB Loan']
    allowed_liabilities = ['Bank Reserves', 'Cash', 'TGA']

    def __init__(self, entry: AccountEntry = AccountEntry(asset={}, liability={})):
        super().__init__('Central Bank', CentralBank.allowed_assets, CentralBank.allowed_liabilities, entry)
        
    def treasury_tranactions(self, counterparty: LiquidityProviders, transaction: str = 'buy', amount: float = 0.0) -> None:
        if isinstance(counterparty, CentralBank):
            print('Central bank cannot trade with itself!')
        elif isinstance(counterparty, Treasury):
            if transaction == 'sell':
                raise ValueError('Central bank cannot sell treasuries to Treasury!')
            elif transaction == 'buy':
                self.asset['Treasury'] += amount
                self.liability['TGA'] += amount
                counterparty.asset['TGA'] += amount
                counterparty.liability['Treasury'] += amount
            else:
                raise ValueError('Transaction has to be buy or sell!')
        else:
            pass


class Treasury(BaseLiquidityProvider):
    allowed_assets = ['TGA']
    allowed_liabilities = ['Treasury']

    def __init__(self, entry: AccountEntry = AccountEntry(asset={}, liability={})):
        super().__init__('Treasury', Treasury.allowed_assets, Treasury.allowed_liabilities, entry)

    def treasury_tranactions(self, counterparty: LiquidityProviders, transaction: str = 'sell', amount: float = 0.0) -> None:
        if isinstance(counterparty, Treasury):
            print('Treasury cannot trade with itself!')
        elif isinstance(counterparty, CentralBank):
            if transaction == 'buy':
                raise ValueError('Treasury cannot buy treasuries from central bank!')
            elif transaction == 'sell':
                self.asset['TGA'] += amount
                self.liability['Treasury'] += amount
                counterparty.liability['TGA'] += amount
                counterparty.asset['Treasury'] +=amount
            else:
                raise ValueError('Transaction has to be buy or sell!')
        else:
            pass
        
   
    
        

NameError: name 'liquidityproviders' is not defined

In [ ]:
cb = CentralBank()     
assert cb.total_asset==0.0


In [ ]:
entry = AccountEntry(asset= {'Treasury':100}, liability={'TGA':100})
cb2 = CentralBank(entry)
assert cb2.total_liability == 100.0

In [ ]:
#test_fail(lambda: CentralBank(AccountEntry(asset= {'Treasury':120}, liability={'TGA':100})),contains = "The sum of asset must be equal to the sume of liability!")
#cb3 = CentralBank(AccountEntry(asset= {'Treasury':120}, liability={'TGA':100}))


In [ ]:
tr=Treasury()
assert tr.total_asset==0.0

In [ ]:
cb2.treasury_tranactions(tr, transaction='buy',amount=150)
test_eq(cb2.total_asset, 250.0)
test_eq(tr.total_asset, 150.0)

In [ ]:
tr.treasury_tranactions(cb2, transaction='sell',amount=110)
test_eq(cb2.total_asset, 360.0)
test_eq(tr.total_asset, 260.0)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()